# Exemples sur DVF+ open-data

In [1]:
import plotly.io as pio
import plotly.express as px
import folium
from folium.plugins import HeatMap

pio.renderers.default = "notebook"

from apifoncier import configure
import apifoncier.dvf_opendata as dvf

In [2]:
# On désactive l'affichage des barres de progression
configure(PROGRESS_BAR=False)

### Distribution des ventes de maison sur une commune

In [7]:
mutations = dvf.mutations(
    code_insee=["59646"], anneemut_min=2017, codtypbien="111", valeurfonc_max=1000000
)
mutations["valeurfonc"] = mutations["valeurfonc"].astype(float)
fig = px.violin(
    mutations,
    y="valeurfonc",
    x="anneemut",
    color="anneemut",
    box=True,
    title=f"Distribution annuelle des prix des ventes de maison à partir de 2017",
    labels={
        "annee": "Année",
        "valeurfonc": "Prix en €",
    },
)
fig.update_layout(legend_title_text="Année de mutation")
fig.show()

### Carte de chaleur des ventes de maisons

In [4]:
# On sélectionne les ventes de maisons après 2017
geomutations = dvf.geomutations(
    code_insee=["62826", "62261"], codtypbien="111", anneemut_min=2017
)

In [5]:
geomutations["centroid"] = geomutations["geometry"].centroid
geomutations["lon"] = geomutations["centroid"].x
geomutations["lat"] = geomutations["centroid"].y

In [6]:
heat_df = geomutations.loc[:, ["lat", "lon", "valeurfonc"]]
heat_df = heat_df.dropna()
heat_df["valeurfonc"] = heat_df["valeurfonc"].astype(float)
heat_data = heat_df.values.tolist()

map = folium.Map(location=[50.5335, 1.60724], zoom_start=13)
HeatMap(heat_data, radius=14, max_zoom=15).add_to(map)
map

### Mutations de foncier nu

In [30]:
# On récupère les mutations de foncier nu de plus de 1ha sur Lille
gdf = dvf.geomutations(code_insee="59350", codtypbien="2", sterr_min=10000)

map = folium.Map(location=[50.63, 3.075], zoom_start=13)
folium.GeoJson(
    gdf.to_json(),
    popup=folium.GeoJsonPopup(
        fields=("sterr", "valeurfonc"), aliases=("Surface (m2)", "Valeur foncière (€)")
    ),
).add_to(map)
map